In [ ]:
#------READ ME------------#
# Suita, Osaka, 09/11/2023
# Rajuli Amra & Yahcut mejambek Van Bakong. 

# Modified every path to your local computer path
# Before using this code, it is advisable to learn the basic of orfeo toolbox algorithm, concept in Object Based Image Classification(OBIA), machine learning and satellite image
# OBIA: https://doi.org/10.1016/j.jag.2020.102263 ; https://doi.org/10.1016/j.isprsjprs.2017.06.001 ; https://doi.org/10.3390/rs14030646 ; https://doi.org/10.1186/s40965-017-0031-6
# OTB: https://github.com/orfeotoolbox/OTB
# python OTB: https://github.com/orfeotoolbox/pyotb

# The Raw input dataset has been pre-processed for radiometric calibration (TOA-BOA) spatial co-registration and radiometric normalization. This step is crucial for time series analysis and change detection. 
# The Norm input was surface reflectance normalize to 0-1 and added 4 spectral indices (NDVI, NDWI, BU2 and ISU)
# Please refer to: https://doi.org/10.3390/rs9070676 ; https://doi.org/10.1016/j.compag.2019.104893 ; https://doi.org/10.1016/j.jag.2020.102290
# Co-registration: https://github.com/GFZ/arosics/tree/main/docs
# Radiometric Normalization: https://github.com/latmperkmol/ts-norm ; https://github.com/swegmueller/LORACCS_Mosaic_Correction (in Input Image is Planetscope)
# If you use this code, please cite our following paper: https://doi.org/10.1016/j.rsase.2024.101438 





In [2]:
#DATASET#

In [5]:

title='Landsat-5 TM 30m 2004'
dataset='Landsat-5-TM'
#Image-Input#
input_raw='/PROCESS/chapter_2_1/input_for_obia/target_landsat_30m.tif'
input_norm='../../input_for_obia/combine_norm/landsat-2004-30m.tif'





In [9]:
#OBJECT BASED-IMAGE CLASSIFICATION (OBIA)#
import pyotb

In [8]:
#Segmentation
#Segmentation using MSS algorithm (https://doi.org/10.1016/j.jag.2020.102263; doi: 10.1109/34.1000236; )
segment_result=f'../SEGMENTATION/{dataset}_segment.shp'
segmen= pyotb.Segmentation({'in'        : input_raw,
                            'filter'    : 'meanshift',
                            })

KeyError: "Segmentation: parameter 'spatialr' was not recognized. Available keys are ('in', 'filter', 'filter.meanshift.spatialr', 'filter.meanshift.ranger', 'filter.meanshift.thres', 'filter.meanshift.maxiter', 'filter.meanshift.minsize', 'filter.cc.expr', 'filter.watershed.threshold', 'filter.watershed.level', 'filter.mprofiles.size', 'filter.mprofiles.start', 'filter.mprofiles.step', 'filter.mprofiles.sigma', 'mode', 'mode.vector.out', 'mode.vector.outmode', 'mode.vector.inmask', 'mode.vector.neighbor', 'mode.vector.stitch', 'mode.vector.minsize', 'mode.vector.simplify', 'mode.vector.layername', 'mode.vector.fieldname', 'mode.vector.tilesize', 'mode.vector.startlabel', 'mode.vector.ogroptions', 'mode.raster.out')"

In [ ]:
#zonal Statistic
zonal_stat=f'../Zonal_stat/{dataset}_zonal_stat.gpkg'
zonal=pyotb.ZonalStatistics({'in'                   : input_norm,
                             'inzone.vector.in'     : segment_result,
                             'out.vector.filename'  : zonal_stat,
                             })

In [153]:
#Point Sample Label
print('For sampling of training and validation of polygon dataset using point label interpretation using input, google earth VHR,input spectral indices (NDVI, NDWI, BI2, ISU) '
'\nand expert assistance is performed in QGIS. Polygon sample is extracted from zonal Statistic output combine with point sample.')

print('Alternatively, you can import pyQGIS and perform the training together with this code. However, sampling requires visual interpretation which is better in QGIS or any GIS application ')

For sampling of training and validation of polygon dataset using point label interpretation using input, google earth VHR,input spectral indices (NDVI, NDWI, BI2, ISU) 
and expert assistance is performed in QGIS. Polygon sample is extracted from zonal Statistic output combine with point sample.
Alternatively, you can import pyQGIS and perform the training together with this code. However, sampling requires visual interpretation which is better in QGIS or any GIS application 


In [ ]:
#Training and Validating Sample Selection


In [6]:
#Train Random Forest
# Accuracy of Random forest algorithm is defined by tree number (classifier.rf.nbtrees), tree depth (classifier.rf.max), min sample leaf (classifier.rf.min),
# and split (classifier.rf.cat). Lower value resulted in noise and low accuracy, while higher value cause high computational time and overfitting. 
# To get optimal combination, Hyperparameter-tuning and cross-validation (CV) is recommended. CV can perform using GridSearchCV algorithm. 
# Hyperparameter-tuning: https://doi.org/10.1007/s10661-015-4489-3 ; https://doi.org/10.3389/feart.2023.1188093 ; https://doi.org/10.1080/01431161.2019.1580825 ;
#                        http://dx.doi.org/10.1016/j.isprsjprs.2016.01.011 ; https://doi.org/10.3390/rs15133241
# GridSearch CV: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


##INPUT#
Training_data='../training_data/training_sample_landsat_70.gpkg'
Validation_data='../training_data/training_sample_landsat_30.gpkg'

#OUTPUT#
model=f'../model/{dataset}_model.txt'
matrix=f'../model/{dataset}_matrix.txt'

kwargs = {
    'feat'                  : ['mean_0','stdev_0','mean_1','stdev_1','mean_2','stdev_2','mean_3','stdev_3', 'mean_4','stdev_4','mean_5','stdev_5','mean_6','stdev_6','mean_7','stdev_7'],
    'cfield'                : ['LC'],
    'classifier'            : 'rf',
    'classifier.rf.max'     : 16,
    'classifier.rf.min'     : 2,
    'classifier.rf.nbtrees' : 500,
    'classifier.rf.ra'      : 0,
    'classifier.rf.cat'     : 5,
    'classifier.rf.var'     : 0,
    }


Model_training=pyotb.TrainVectorClassifier({'io.vd'             : Training_data,
                                            'io.out'            : model,
                                            **kwargs,
                                            'io.confmatout'     : matrix,
                                            'valid.vd'          : Validation_data,
                                            })


2023-11-10 11:44:38 (INFO) [pyOTB] TrainVectorClassifier: argument for parameter "io.vd" was converted to list
2023-11-10 11:44:38 (INFO) [pyOTB] TrainVectorClassifier: argument for parameter "valid.vd" was converted to list
2023-11-10 11:44:38 (INFO) TrainVectorClassifier: Reading vector file 1/1
2023-11-10 11:44:38 (INFO) TrainVectorClassifier: Reading vector file 1/1
2023-11-10 11:44:38 (INFO) TrainVectorClassifier: Computing model file : ../model/Landsat-5_model.txt
Training model...: 100% [**************************************************] (3s)
Validation...: 100% [**************************************************] (0s)
2023-11-10 11:44:42 (INFO) TrainVectorClassifier: Predicted list size : 649
2023-11-10 11:44:42 (INFO) TrainVectorClassifier: ValidationLabeledListSample size : 649
2023-11-10 11:44:42 (INFO) TrainVectorClassifier: Training performances:
2023-11-10 11:44:42 (INFO) TrainVectorClassifier: Confusion matrix (rows = reference labels, columns = produced labels):
    [1

In [9]:
#ACCURACY ASSESSMENT#
import csv
import numpy as np
import pandas as pd
from tabulate import tabulate

In [10]:
#confusion_matrix
matrix = np.loadtxt(matrix, delimiter=",")
matrix = matrix.astype(int)

confusion_matrix = np.array(matrix)
print(confusion_matrix)



[[109   0   0   0   0   1   0]
 [  0  30   0   0   0   0   0]
 [  0   0 129   0   0   8   0]
 [  0   0   0  53   0   0   0]
 [  0   0   0   0   5   8   0]
 [  0   0   0   0   0 299   0]
 [  0   0   0   0   0   0   7]]


In [11]:
#Accuracy Calculation#

TP = np.diag(confusion_matrix)
FP = confusion_matrix.sum(axis=0) - TP
FN = confusion_matrix.sum(axis=1) - TP
TN = confusion_matrix.sum() - (TP + FP + FN)

#Producer, User and Overall Accuracy

producer_accuracy = TP / confusion_matrix.sum(axis=1)
user_accuracy = TP / confusion_matrix.sum(axis=0)
overall_accuracy = (TP.sum()) / (TP.sum() + FP.sum())

print("--------------------------------")
print("Producer Accuracy:", " ".join([f"{x*100:.1f}%" for x in producer_accuracy]))
print("User Accuracy:", " ".join([f"{x*100:.1f}%" for x in user_accuracy]))
print(f"OA: {overall_accuracy*100:.1f}%")

#Precision, Recall, F1-Score

precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1_score=(2 * precision * recall / (precision + recall))
F1_macro = np.mean(2 * precision * recall / (precision + recall))

print("--------------------------------")
print("Precision:", " ".join([f"{x:.2f}" for x in precision]))
print("Recall:", " ".join([f"{x:.2f}" for x in recall]))
print("F1:", " ".join([f"{x:.2f}" for x in F1_score]))
print(f"F1_macro: {F1_macro:.2f}")

#Kappa index

n = confusion_matrix.sum()
Po = np.diag(confusion_matrix).sum() / n
Pe = (confusion_matrix.sum(axis=0) * confusion_matrix.sum(axis=1)).sum() / (n ** 2)
kappa = (Po - Pe) / (1 - Pe)

print("--------------------------------")
print(f"Kappa Index: {kappa:.2f}")

#Save Accuracy as Table

table = [
    ["Class", "A", "B", "C", "D", "E", "F","G"],
    ["Precision", *[f"{x:.2f}" for x in precision]],
    ["Recall", *[f"{x:.2f}" for x in recall]],
    ["F1 Score", *[f"{x:.2f}" for x in F1_score]],
    ["User Accuracy", *[f"{x*100:.1f}%" for x in user_accuracy]],
    ["Producer Accuracy", *[f"{x*100:.1f}%" for x in producer_accuracy]],
    ["Overall Accuracy", "", "", "", "", "", f"{overall_accuracy*100:.1f}%"],
    ["F1 Macro", "", "", "", "", "", f"{F1_macro:.2f}"],
    ["Kappa Index", "", "", "", "", "", f"{kappa:.2f}"]
]

accuracy_metrics=f'../result/{dataset}_accuracy.txt'
with open(accuracy_metrics, "w",newline='') as f:
    writer = csv.writer(f)
    writer.writerows(table)    
    
df = pd.read_csv(accuracy_metrics, header=None)
df = df.dropna(how='all')
df = df.dropna(axis=1, how='all')
df = df.fillna('')


#print(df.to_string(index=False, header=False))
print(tabulate(df, headers='firstrow', tablefmt='psql'))

--------------------------------
Producer Accuracy: 99.1% 100.0% 94.2% 100.0% 38.5% 100.0% 100.0%
User Accuracy: 100.0% 100.0% 100.0% 100.0% 100.0% 94.6% 100.0%
OA: 97.4%
--------------------------------
Precision: 1.00 1.00 1.00 1.00 1.00 0.95 1.00
Recall: 0.99 1.00 0.94 1.00 0.38 1.00 1.00
F1: 1.00 1.00 0.97 1.00 0.56 0.97 1.00
F1_macro: 0.93
--------------------------------
Kappa Index: 0.96
+-----+-------------------+--------+--------+--------+--------+--------+--------+--------+
|   0 | Class             | A      | B      | C      | D      | E      | F      | G      |
|-----+-------------------+--------+--------+--------+--------+--------+--------+--------|
|   1 | Precision         | 1.00   | 1.00   | 1.00   | 1.00   | 1.00   | 0.95   | 1.00   |
|   2 | Recall            | 0.99   | 1.00   | 0.94   | 1.00   | 0.38   | 1.00   | 1.00   |
|   3 | F1 Score          | 1.00   | 1.00   | 0.97   | 1.00   | 0.56   | 0.97   | 1.00   |
|   4 | User Accuracy     | 100.0% | 100.0% | 100.0% | 1

In [14]:
#Classification#
import geopandas as gpd

In [29]:
import xml.etree.ElementTree as ET

# Your XML data
xml_data = '../input/landsat-2004-30m.xml'

# Read the XML data from the file
with open(xml_data, 'r') as file:
    xml_data = file.read()

root = ET.fromstring(xml_data)

# Iterate through each 'Statistic' element
for statistic in root.findall('.//Statistic'):
    values = []

    # Iterate through 'StatisticVector' elements within each 'Statistic'
    for statistic_vector in statistic.findall('.//StatisticVector'):
        values.append(statistic_vector.get('value'))

    # Remove the existing 'StatisticVector' elements
    for statistic_vector in statistic.findall('.//StatisticVector'):
        statistic.remove(statistic_vector)

    # Create a new 'StatisticVector' element and set its 'value' attribute
    new_statistic_vector = ET.SubElement(statistic, 'StatisticVector')
    new_statistic_vector.set('value', '[' + ', '.join(values) + ']')

# Save the modified XML to a new file
new_xml_file = '../input/new_xml_file.xml'
with open(new_xml_file, 'w') as file:
    file.write(ET.tostring(root, encoding='unicode'))

print(new_xml_file)

../input/new_xml_file.xml


In [32]:
import rasterio
import xml.etree.ElementTree as ET

# Open the satellite image
with rasterio.open(input_norm) as src:

    # Compute the image statistics
    stats = src.read().flatten()
    mean = stats.mean()
    std = stats.std()
    min = stats.min()
    max = stats.max()

# Create a new XML file with the computed statistics
root = ET.Element("FeatureStatistics")
mean_elem = ET.SubElement(root, "Statistic", name="mean")
stddev_elem = ET.SubElement(root, "Statistic", name="stddev")
min_elem = ET.SubElement(root, "Statistic", name="min")
max_elem = ET.SubElement(root, "Statistic", name="max")
ET.SubElement(mean_elem, "StatisticVector", value=str(mean))
ET.SubElement(stddev_elem, "StatisticVector", value=str(std))
ET.SubElement(min_elem, "StatisticVector", value=str(min))
ET.SubElement(max_elem, "StatisticVector", value=str(max))

# Write the new XML file
tree = ET.ElementTree(root)
tree.write("../input/file_tes.xml")

In [13]:

#Post-processing is sometime needed to reduce misclassification between sand class and built-up class (doi:10.3390/rs11030345; https://doi.org/10.3390/atmos14020240 )

zonal_stat='../Zonal_stat/Landsat_2004_30_zonal_stat_fix.gpkg'
shapefile_output=f'../result/{dataset}_vector.shp'
classification=pyotb.VectorClassifier({'in'     : zonal_stat,
                                       'model'  : model,
                                       'feat'   : ['mean_0','stdev_0','mean_1','stdev_1','mean_2','stdev_2','mean_3','stdev_3', 'mean_4','stdev_4','mean_5','stdev_5','mean_6','stdev_6','mean_7','stdev_7'],
                                       'cfield' : 'land_type',
                                       'out'    : shapefile_output,
                                       })

2023-11-10 11:48:18 (INFO) VectorClassifier: Model loaded
2023-11-10 11:48:18 (INFO) VectorClassifier: mean used: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2023-11-10 11:48:18 (INFO) VectorClassifier: standard deviation used: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2023-11-10 11:48:18 (INFO) VectorClassifier: Loading model


In [21]:
#select only land cover type as new vector

LC= gpd.read_file(shapefile_output)
print(LC.columns)

selected_field='land_type'

dissolved = LC.dissolve(by=selected_field)
LC_vector = f'../result/{dataset}_LC_vector.shp'

dissolved.to_file(LC_vector)

Index(['label', 'meanB0', 'meanB1', 'meanB2', 'meanB3', 'varB0', 'varB1',
       'varB2', 'varB3', 'count', 'mean_0', 'stdev_0', 'min_0', 'max_0',
       'mean_1', 'stdev_1', 'min_1', 'max_1', 'mean_2', 'stdev_2', 'min_2',
       'max_2', 'mean_3', 'stdev_3', 'min_3', 'max_3', 'mean_4', 'stdev_4',
       'min_4', 'max_4', 'mean_5', 'stdev_5', 'min_5', 'max_5', 'mean_6',
       'stdev_6', 'min_6', 'max_6', 'mean_7', 'stdev_7', 'min_7', 'max_7',
       'land_type', 'geometry'],
      dtype='object')


In [22]:
#convert to raster_map

all_class=f'../result/{dataset}_LC_all_class.tif'

pyotb.Rasterization({'in'                   : LC_vector,
                     'out'                  : all_class,
                     'im'                   : input_norm,
                     'mode'                 : 'attribute',
                     'mode.attribute.field' : 'land_type',                     
                     })

2023-11-10 12:15:00 (INFO) Rasterization: Input dataset extent is (751665, 610095) (763665, 621195)
2023-11-10 12:15:00 (INFO) Rasterization: Input dataset projection reference system is: PROJCS["WGS 84 / UTM zone 46N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32646"]]
2023-11-10 12:15:00 (INFO): Loading metadata from official product
2023-11-10 12:15:00 (INFO) Rasterization: Output projection reference system is: PROJCS["WGS 84 / UTM zone 46N",GEOGCS["WGS 84",DATUM["WGS_1984",SP

<pyotb.Rasterization object, id 2008366678344>